In [1]:
import torch
import torch.nn as nn
from torchsnooper import snoop

In [2]:
class SA(nn.Module):
    def __init__(self, ndim, k_dim, v_dim):
        super(SA,self).__init__()
        self.q1 = nn.Conv1d(ndim,k_dim,1)  # channel数就是词向量的维度，kernel_size = 1的意思是时间维度不变，也就是词的数量
        self.k1 = nn.Conv1d(ndim,k_dim,1)
        self.v1 = nn.Conv1d(ndim,v_dim,1)
    @snoop()
    def forward(self,x):
        q = self.q1(x)
        k = self.k1(x)
        v = self.v1(x)
        o1 = torch.matmul(k.permute(0,2,1),q).softmax(1)
        o2 = torch.matmul(v,o1)
        return o2

* 这个其实应该是没有scaled的, 不能叫scaled dot product attention 因为它没有除以sqrt(K)

In [3]:
x = torch.rand(42, 512, 135)
model = SA(512,1024,512)
out = model(x)

Source path:... <ipython-input-2-9be4942b5083>
Starting var:.. self = SA(  (q1): Conv1d(512, 1024, kernel_size=(1,), s... Conv1d(512, 512, kernel_size=(1,), stride=(1,)))
Starting var:.. x = tensor<(42, 512, 135), float32, cpu>
15:11:30.900458 call         8     def forward(self,x):
15:11:30.905356 line         9         q = self.q1(x)
New var:....... q = tensor<(42, 1024, 135), float32, cpu, grad>
15:11:31.411780 line        10         k = self.k1(x)
New var:....... k = tensor<(42, 1024, 135), float32, cpu, grad>
15:11:31.903031 line        11         v = self.v1(x)
New var:....... v = tensor<(42, 512, 135), float32, cpu, grad>
15:11:32.169166 line        12         o1 = torch.matmul(k.permute(0,2,1),q).softmax(1)
New var:....... o1 = tensor<(42, 135, 135), float32, cpu, grad>
15:11:32.197913 line        13         o2 = torch.matmul(v,o1)
New var:....... o2 = tensor<(42, 512, 135), float32, cpu, grad>
15:11:32.216209 line        14         return o2
15:11:32.231383 return      14     

In [4]:
out.shape

torch.Size([42, 512, 135])